In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("amazon/Reviews.csv")

In [3]:
amzn = data[["Score", "Text"]]
amzn.columns = ["stars", "text"]
amzn["source"] = "Amazon"

/Users/gregpollock/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
amzn.head()

,stars,text,source
0,5,I have bought several of the Vitality canned d...,Amazon
1,1,Product arrived labeled as Jumbo Salted Peanut...,Amazon
2,4,This is a confection that has been around a fe...,Amazon
3,2,If you are looking for the secret ingredient i...,Amazon
4,5,Great taffy at a great price. There was a wid...,Amazon


In [5]:
data = pd.read_csv("yelp.csv")

In [6]:
yelp = data[["stars", "text"]]
yelp["source"] = "Yelp"

/Users/gregpollock/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
yelp.head()

,stars,text,source
0,5,My wife took me here on my birthday for breakf...,Yelp
1,5,I have no idea why some people give bad review...,Yelp
2,4,love the gyro plate. Rice is so good and I als...,Yelp
3,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",Yelp
4,5,General Manager Scott Petello is a good egg!!!...,Yelp


In [8]:
df = amzn.append(yelp).reset_index(drop = True)

In [9]:
display(df)

,stars,text,source
0,5,I have bought several of the Vitality canned d...,Amazon
1,1,Product arrived labeled as Jumbo Salted Peanut...,Amazon
2,4,This is a confection that has been around a fe...,Amazon
3,2,If you are looking for the secret ingredient i...,Amazon
4,5,Great taffy at a great price. There was a wid...,Amazon
...,...,...,...
578449,3,First visit...Had lunch here today - used my G...,Yelp
578450,4,Should be called house of deliciousness!\n\nI ...,Yelp
578451,4,I recently visited Olive and Ivy for business ...,Yelp
578452,2,My nephew just moved to Scottsdale recently so...,Yelp


In [10]:
from nltk import wordpunct_tokenize, word_tokenize, sent_tokenize
from nltk.corpus import stopwords

In [11]:
tokens = df['text'].apply(lambda x: wordpunct_tokenize(x))
df['tokens'] = tokens

In [12]:
mini = df.iloc[0:10,:]

In [13]:
mini

,stars,text,source,tokens
0,5,I have bought several of the Vitality canned d...,Amazon,"[I, have, bought, several, of, the, Vitality, ..."
1,1,Product arrived labeled as Jumbo Salted Peanut...,Amazon,"[Product, arrived, labeled, as, Jumbo, Salted,..."
2,4,This is a confection that has been around a fe...,Amazon,"[This, is, a, confection, that, has, been, aro..."
3,2,If you are looking for the secret ingredient i...,Amazon,"[If, you, are, looking, for, the, secret, ingr..."
4,5,Great taffy at a great price. There was a wid...,Amazon,"[Great, taffy, at, a, great, price, ., There, ..."
5,4,I got a wild hair for taffy and ordered this f...,Amazon,"[I, got, a, wild, hair, for, taffy, and, order..."
6,5,This saltwater taffy had great flavors and was...,Amazon,"[This, saltwater, taffy, had, great, flavors, ..."
7,5,This taffy is so good. It is very soft and ch...,Amazon,"[This, taffy, is, so, good, ., It, is, very, s..."
8,5,Right now I'm mostly just sprouting this so my...,Amazon,"[Right, now, I, ', m, mostly, just, sprouting,..."
9,5,This is a very healthy dog food. Good for thei...,Amazon,"[This, is, a, very, healthy, dog, food, ., Goo..."


In [14]:
sw = stopwords.words('english')

In [15]:
mini['tokens'] = mini['tokens'].apply(lambda x: [word for word in x if word not in sw])

/Users/gregpollock/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
df['tokens'] = df['tokens'].apply(lambda x: [word for word in x if word not in sw])

In [17]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [18]:
sid = SentimentIntensityAnalyzer()

In [19]:
df

,stars,text,source,tokens
0,5,I have bought several of the Vitality canned d...,Amazon,"[I, bought, several, Vitality, canned, dog, fo..."
1,1,Product arrived labeled as Jumbo Salted Peanut...,Amazon,"[Product, arrived, labeled, Jumbo, Salted, Pea..."
2,4,This is a confection that has been around a fe...,Amazon,"[This, confection, around, centuries, ., It, l..."
3,2,If you are looking for the secret ingredient i...,Amazon,"[If, looking, secret, ingredient, Robitussin, ..."
4,5,Great taffy at a great price. There was a wid...,Amazon,"[Great, taffy, great, price, ., There, wide, a..."
...,...,...,...,...
578449,3,First visit...Had lunch here today - used my G...,Yelp,"[First, visit, ..., Had, lunch, today, -, used..."
578450,4,Should be called house of deliciousness!\n\nI ...,Yelp,"[Should, called, house, deliciousness, !, I, c..."
578451,4,I recently visited Olive and Ivy for business ...,Yelp,"[I, recently, visited, Olive, Ivy, business, l..."
578452,2,My nephew just moved to Scottsdale recently so...,Yelp,"[My, nephew, moved, Scottsdale, recently, bunc..."


In [20]:
df['scores'] = df['tokens'].apply(lambda x: sid.polarity_scores(" ".join(x))['compound'])

In [21]:
mini['scores'] = mini['tokens'].apply(lambda x: sid.polarity_scores(" ".join(x))['compound'])

/Users/gregpollock/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
df

,stars,text,source,tokens,scores
0,5,I have bought several of the Vitality canned d...,Amazon,"[I, bought, several, Vitality, canned, dog, fo...",0.9413
1,1,Product arrived labeled as Jumbo Salted Peanut...,Amazon,"[Product, arrived, labeled, Jumbo, Salted, Pea...",0.0762
2,4,This is a confection that has been around a fe...,Amazon,"[This, confection, around, centuries, ., It, l...",0.8073
3,2,If you are looking for the secret ingredient i...,Amazon,"[If, looking, secret, ingredient, Robitussin, ...",0.4404
4,5,Great taffy at a great price. There was a wid...,Amazon,"[Great, taffy, great, price, ., There, wide, a...",0.9468
...,...,...,...,...,...
578449,3,First visit...Had lunch here today - used my G...,Yelp,"[First, visit, ..., Had, lunch, today, -, used...",0.8807
578450,4,Should be called house of deliciousness!\n\nI ...,Yelp,"[Should, called, house, deliciousness, !, I, c...",0.9925
578451,4,I recently visited Olive and Ivy for business ...,Yelp,"[I, recently, visited, Olive, Ivy, business, l...",0.9937
578452,2,My nephew just moved to Scottsdale recently so...,Yelp,"[My, nephew, moved, Scottsdale, recently, bunc...",-0.9491


In [23]:
from sklearn.model_selection import train_test_split

In [59]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

In [25]:
train, test = train_test_split(df, test_size=.3, stratify=df.stars, random_state=314)

In [26]:
train.head()

,stars,text,source,tokens,scores
316916,3,I love Fig Newtons and I love crisp cookies. ...,Amazon,"[I, love, Fig, Newtons, I, love, crisp, cookie...",0.9657
371462,5,These are made in the USA which is the first t...,Amazon,"[These, made, USA, first, thing, I, look, buyi...",0.6808
519439,5,"I absolutely love this k cup coffee,i've tried...",Amazon,"[I, absolutely, love, k, cup, coffee, ,, ', tr...",0.8832
65441,3,The first thing you'll notice about these pepp...,Amazon,"[The, first, thing, ', notice, peppered, cashe...",0.9864
204707,5,Taste exactly like Atlanta Bread's Kona Coffee...,Amazon,"[Taste, exactly, like, Atlanta, Bread, ', Kona...",0.8302


In [56]:
np.array(train['scores']).reshape(-1, 1)

array([[ 0.9657],
       [ 0.6808],
       [ 0.8832],
       ...,
       [ 0.1027],
       [ 0.7717],
       [-0.6494]])

In [43]:
import numpy as np

In [58]:
np.shape(train['scores'])

(404917,)

In [61]:
rfc = RandomForestClassifier()

In [29]:
nb = MultinomialNB()

In [62]:
rfc.fit(np.array(train['scores']).reshape(-1, 1), train['stars'])

RandomForestClassifier()

In [66]:
yhat = rfc.predict(np.array(train['scores']).reshape(-1, 1))

In [67]:
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, roc_auc_score, roc_curve, accuracy_score

In [72]:
yhat == test['stars']

/Users/gregpollock/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


ValueError: Lengths must match to compare

In [71]:
#print(accuracy_score(np.array(train['scores']).reshape(-1, 1), yhat))
print(f1_score(np.array(train['scores']).reshape(-1, 1), yhat))
#print(precision_score(np.array(train['scores']).reshape(-1, 1), yhat))
#print(recall_score(np.array(train['scores']).reshape(-1, 1), yhat))

ValueError: Classification metrics can't handle a mix of continuous and multiclass targets

In [36]:
yhat

array([5, 5, 5, ..., 5, 5, 5])

In [38]:
173537 / 404917

0.4285742510193447